# Хэширование в строковых задачах

Хэширование — это сопоставление некоторым объектам числовых значений. Оно обычно не обратно однозначное: многим хэшу может соответствовать много объектов. Иначе говоря, хэш — это просто функция, но в каком-то смысле «детерминированно случайная».

Объекты могут быть самыми разными: тексты, изображения, графы, шахматные позиции и просто битовые файлы. Мы остановимся на строках.

Алгоритмы, основанные на хешировании подстрок, позволяют решить огромное число задач (см. примеры). 

**Лайфхак**: пока вы не выучили все детерминированные строковые алгоритмы, научитесь пользоваться хэшами.

Есть одино интересное семейство хэшей: полиномиальные.

Давайте представим строку в виде последовательности чисел $\{a_i\}$, каждое лежит от 1 до размера алфавита. Тогда хэш от неё будем считать двумя вариантами:

1. $h_1 = (a_0) \mod p$ 
2. $h_2 = $

Здесь $k$ — произвольное число большее размера алфавита, а $p$ — достаточно большой модуль, вообще говоря, не обязательно простой.

In [ ]:
const int k = 31, mod = 1e9+7;

string s = "abacabadaba";
long long h = 0;
for (char x : s)
    h = (h*k + x + 1) % mod;

В чем, собственно преимущество: чтобы посчитать хэш подстроки, достаточно посчитать префиксные хэши, а затем делать быстрые операции вычитания и умножения. Это позволяет сравнивать строки за $O(1)$.

Давайте вместо приведения к нулевым степяням приведем их к каким-то одинаковым — например, к $n$-ной. Так код упростится.

In [ ]:
typedef unsigned long long ull;

ull h[n];


bool hash (int l, int r) {
    return p[n-r] * (h[r] - h[l]);
}

// ...

for (int i = 0; i < n; i++)
    p[i] = p[i-1]*k;

for (int i = 0; i < n; i++) {
    h[i] = h[i-1] + s[i] * p[i];
}

# Примеры задач

**Количество разных подстрок**. Можно посчитать хэши от всех подстрок за $O(n^2)$ и запихнуть их все в set, а затем сделать `s.size()`.

**Поиск подстроки в строке**. Можно посчитать хэши от паттерна и пройтись «окном» по тексту, начитывая хэш окна. Это называют алгоритмом Рабина-Карпа.

**Сравнение строк**. У любых двух строк есть какой-то (возможно, пустой) общий префикс. Можно сделать бинпоиск по его длине, а дальше сравнить два символа, идущие за ним.

**Проверка на палиндром**. Можно посчитать два массива — обратные хэши и прямые. Проверка на палиндром будет заключаться в сравнении соответствующих разностей префиксных сумм.

**Количество палиндромов**. Можно перебрать центр палиндрома, а для каждого центра — его размер. Проверять на палиндромность подстроки мы уже умеем.

# Вероятность ошибки и выбор констант

 У алгоритмов, используюзих хэширование, есть один существенный недостаток: недетерминированность. На CodeForces даже иногда бывали случаи взломов, сгенерированных против конкретных значений параметров.

**Парадокс дней рождений**. Если в классе есть 22 учеников, то с вероятностью более 50% у кого-то совпадут дни рождения. В общем случае он формулируется так: если есть $n$ чисел от 1 до $n^2$, то вероятность совпадения не очень большая и не очень маленькая. Доказывать это мы не будем, по крайней мере сейчас.

Полезное правило: брать модуль на несколько порядков больше, чем квадрат количества сравнений.

Можно брать два или даже три модуля.

Можно также брать модуль $2^{64}$. Он удобен тем, что не о каких переполнениях заботиться не нужно: можно просто все хранить в `unsigned long long` — архитектура процессора устроена так, что сама перенесет лишнее. Это может работать очень быстро. Существует известный тест против него, однако на некоторых контестах его могут не добавить.

В выборе $k$ и $p$ нет никакой логики. Главное, чтобы их не знал человек, генерирующий тесты. Имеет смысл, чтобы они были взаимно простыми, чтобы начианя с какого-то момента все не обнулилось.